### Guide to the notebook

__The notebook in a nutshell:__ Imported the aggregated DF and applied Random Forest thrice using different Hyperparameters and noted the RMSE on the validation as well as the test set. Next, Linear Regression was applied, and lastly, a small Neural Net was applied. Since the validation and test scores formed around about a straight line (their variance was similar), the chosen validation set was exported for future use.

### Importing Libs

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from sklearn import svm

Using TensorFlow backend.


### Importing & Fixing Datasets

In [2]:
Xtrain = pd.read_pickle('XtrainRF'); Ytrain = pd.read_pickle('YtrainRF'); Xtest = pd.read_pickle('XtestRF')

Xtrain[['feature_1', 'feature_2']] = Xtrain[['feature_1', 'feature_2']].astype('category')

Xtrain[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']]\
=Xtrain[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']].astype('int32')

Xtest[['feature_1', 'feature_2']] = Xtest[['feature_1', 'feature_2']].astype('category')

Xtest[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']]\
=Xtest[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']].astype('int32')

### Creating Vars

In [11]:
# Make Vars
X=Train.loc[:,Train.columns!='target']
Y=Train.loc[:,'target']

Xtrain,Xval,Ytrain,Yval=train_test_split(Xtrain,Ytrain,test_size=0.20)

Xtrain.to_pickle('Xtrain');Xval.to_pickle('Xval');Ytrain.to_pickle('Ytrain');Yval.to_pickle('Yval')

Xtrain.first_active_month=Xtrain.first_active_month.astype('category');Xtrain.first_active_month=Xtrain.first_active_month.cat.codes;Xtrain

Xval.first_active_month=Xval.first_active_month.astype('category');Xval.first_active_month=Xval.first_active_month.cat.codes;Xval

XtrainRf1=Xtrain.loc[:,Xtrain.columns!='card_id'];XvalRf1=Xval.loc[:,Xval.columns!='card_id']

Xtest=Test.loc[:,Test.columns!='card_id']

Xtest.first_active_month=Xtest.first_active_month.astype('category').cat.codes;Xtest.describe(include='all')

### Applying 1st RF

In [16]:
RF=RandomForestRegressor(n_estimators=100,max_features=0.5,min_samples_leaf=1,bootstrap=False,n_jobs=-1)

modelRF=RF.fit(Xtrain,Ytrain)

RF1.oob_score_

Ypred=RF.predict(Xval)

#### RMSE of val set

In [7]:
# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.714320993586702

In [13]:
# RMSE 2nd Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.704414994858926

In [19]:
# RMSE 3rd Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.702026929565763

#### RMSE of test set

In [20]:
Ypred=RF.predict(Xtest)

In [21]:
Ypred=pd.Series(Ypred);Ypred.to_csv('Ypred.csv')

C:\Users\AMS\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


__RMSE Test Set 1st Check: 3.97082__

__RMSE Test Set 2nd Check: 3.96522__

__RMSE Test Set 3rd Check: 3.96388__

### Applying LinReg

#### One Hot Encoding

In [ ]:
XtrainLR=pd.get_dummies(Xtrain,columns=['feature_1','feature_2','feature_3'],drop_first=True);XtrainLR

In [ ]:
XvalLR=pd.get_dummies(Xval, columns=['feature_1','feature_2','feature_3'],drop_first=True);XvalLR

In [ ]:
XtestLR=pd.get_dummies(Xtest,columns=['feature_1','feature_2','feature_3'],drop_first=True);XtestLR

#### Fitting model

In [25]:
LR=LinearRegression(n_jobs=-1)

In [26]:
model=LR.fit(XtrainLR,Ytrain)

In [27]:
Ypred=LR.predict(XvalLR)

#### RMSE of val set

In [28]:
# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.6704172723870974

#### RMSE of test set

In [29]:
Ypred=LR.predict(XtestLR)

In [ ]:
Ypred=pd.Series(Ypred);Ypred.to_csv('Ypred.csv')

__RMSE Test Set 1st Check: 3.94779__

### Applying NN

In [42]:
# introduces randomness. 
# fix random seed for reproducibility
np.random.seed(7)
# load pima indians dataset

# split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(100, input_dim=27, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['mse']) #metrics should be diff becuase of skewed dataset
# Fit the model
model.fit(XtrainLR, Ytrain, epochs=100, batch_size=10000)
# evaluate the model
scores = model.evaluate(XtrainLR, Ytrain)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Ypred=model.predict(XvalLR)

# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

Epoch 1/100
159684/159684 [==============================] - 1s 8us/step - loss: 30316.2305 - mean_squared_error: 30316.2305
Epoch 2/100
159684/159684 [==============================] - 1s 4us/step - loss: 684.7337 - mean_squared_error: 684.7337
Epoch 3/100
159684/159684 [==============================] - 1s 4us/step - loss: 176.3462 - mean_squared_error: 176.3462
Epoch 4/100
159684/159684 [==============================] - 1s 4us/step - loss: 77.8561 - mean_squared_error: 77.8561
Epoch 5/100
159684/159684 [==============================] - 1s 3us/step - loss: 49.4828 - mean_squared_error: 49.4828
Epoch 6/100
159684/159684 [==============================] - 1s 3us/step - loss: 36.3288 - mean_squared_error: 36.3288
Epoch 7/100
159684/159684 [==============================] - 1s 3us/step - loss: 31.7011 - mean_squared_error: 31.7011
Epoch 8/100
159684/159684 [==============================] - 1s 3us/step - loss: 27.5002 - mean_squared_error: 27.5002
Epoch 9/100
159684/159684 [===========

#### RMSE of test set

In [45]:
Ypred=model.predict(XtestLR)

Ypred=pd.DataFrame(Ypred);Ypred.to_csv('Ypred.csv')

__RMSE Test Set 1st Check: 3.95983__

__RMSE Test Set 2nd Check: 16.26424__

__RMSE Test Set 3rd Check: 5.92523__

__RMSE Test Set 4th Check: 5.61297__

### Exporting Data

In [7]:
Xtrain.to_pickle('XtrainRFM16');Xval.to_pickle('XvalRFM16');Xtest.to_pickle('XtestRFM16')

XtrainLR.to_pickle('XtrainLRM16');XvalLR.to_pickle('XvalLRM16');XtestLR.to_pickle('XtestLRM16')

Ytrain.to_pickle('YtrainRFM16');Yval.to_pickle('YvalRFM16')